# [H2] Water – [04] Zouttoevoegingen & stijlgebaseerde waterdesign

**Doel.** Dit notebook helpt u om vanuit een startwater (of een blend uit Notebook 3) met **zouttoevoegingen** een **doelwaterprofiel per bierstijl** te benaderen.

**Wat u hier leert**
- (1) Omrekening van **gram zout → ppm (mg/L) ionen**
- (2) Formuleren van waterdesign als een **lineair optimalisatieprobleem**
- (3) Een eenvoudige **niet-negatieve optimizer** (zonder extra libraries)

> **Scope (belangrijk).** Dit is een onderwijsmodel: we modelleren enkel **ionenconcentraties**.  
> We negeren o.a. (i) ion-activiteit/ionsterkte, (ii) neerslag (bv. CaCO₃), (iii) pH/CO₂-evenwichten en (iv) mout-specifieke buffering.  
> Gebruik dit als **ontwerpstartpunt**, niet als absolute waarheid.


## 1. Theorie (kort)

### 1.1 Massabalans voor ionen (ppm)
In waterchemie is 1 mg/L ≈ 1 ppm (voor verdunde oplossingen).  
Een zouttoevoeging verandert een ionconcentratie lineair:

\[
c_{new} = c_{start} + \sum_j a_j\,\Delta c_j(a_j)
\]

waarbij \(a_j\) de dosis is (g/L of g/20 L), en \(\Delta c_j\) de bijdrage per zout.

### 1.2 Zouten en hun ionbijdragen
Voor elk zout rekenen we de massa-fractie van relevante ionen uit vanuit de molmassa.  
Voorbeeld: CaSO₄·2H₂O (gypsum) levert **Ca²⁺** en **SO₄²⁻**.

### 1.3 Doelprofielen per stijl
Bierstijlen corresponderen vaak met typische chloride/sulfaat-voorkeuren en Ca-niveaus.  
We werken met **streefwaarden** (targets) en minimaliseren een gewogen fout.


## 2. Parameters

1) Kies of definieer een **startwater** (ppm).  
2) Kies een **bierstijl preset** of geef uw eigen targetprofiel.  
3) Kies welke zouten u toelaat.

**Opmerking.** HCO₃⁻ verhogen via NaHCO₃ is didactisch handig. In de praktijk hangt alkaliniteit sterk samen met pH/CO₂.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

IONS = ["Ca","Mg","Na","Cl","SO4","HCO3"]  # mg/L (ppm)

# ===== Startwater (ppm) =====
# (U kunt dit vervangen door output van Notebook 3)
start_water = {"Ca":35, "Mg":8, "Na":15, "Cl":30, "SO4":40, "HCO3":120}

# ===== Stijl-presets (ppm) =====
# Representatieve onderwijs-streefwaarden (niet universeel)
style_presets = {
    "Pilsner (crisp)": {"Ca":50, "Mg":8,  "Na":10, "Cl":40, "SO4":80,  "HCO3":30},
    "Hoppy IPA":       {"Ca":80, "Mg":10, "Na":20, "Cl":50, "SO4":150, "HCO3":40},
    "Malt-forward":    {"Ca":70, "Mg":10, "Na":25, "Cl":120,"SO4":60,  "HCO3":80},
    "Stout (dark)":    {"Ca":90, "Mg":10, "Na":30, "Cl":80, "SO4":80,  "HCO3":180},
}

chosen_style = "Hoppy IPA"  # kies uit style_presets
target = style_presets[chosen_style].copy()

# ===== Wegingen (belangrijkheid per ion) =====
weights = {"Ca":1, "Mg":0.8, "Na":0.4, "Cl":1, "SO4":1, "HCO3":0.8}

chosen_style, target

## 3. Zoutenbibliotheek (didactisch)

We gebruiken veelgebruikte brouwzouten:
- **CaCl₂·2H₂O** (calciumchloride)
- **CaSO₄·2H₂O** (gypsum)
- **MgSO₄·7H₂O** (epsom)
- **NaCl** (keukenzout)
- **NaHCO₃** (baksoda)

Een zout heeft een **bijdragevector** (ppm per g/L) voor de ionen in `IONS`.


In [ ]:
# Atomaire/moleculaire massa's (g/mol)
M = {
    "Ca": 40.078,
    "Mg": 24.305,
    "Na": 22.990,
    "Cl": 35.45,
    "S":  32.065,
    "O":  15.999,
    "H":  1.008,
    "C":  12.011
}

M_SO4  = M["S"] + 4*M["O"]
M_HCO3 = M["H"] + M["C"] + 3*M["O"]

salts = {
    "CaCl2·2H2O": {"MM": M["Ca"] + 2*M["Cl"] + 2*(2*M["H"] + M["O"]),
                   "ions": {"Ca": M["Ca"], "Cl": 2*M["Cl"]}},
    "CaSO4·2H2O": {"MM": M["Ca"] + M_SO4 + 2*(2*M["H"] + M["O"]),
                   "ions": {"Ca": M["Ca"], "SO4": M_SO4}},
    "MgSO4·7H2O": {"MM": M["Mg"] + M_SO4 + 7*(2*M["H"] + M["O"]),
                   "ions": {"Mg": M["Mg"], "SO4": M_SO4}},
    "NaCl":       {"MM": M["Na"] + M["Cl"],
                   "ions": {"Na": M["Na"], "Cl": M["Cl"]}},
    "NaHCO3":     {"MM": M["Na"] + M_HCO3,
                   "ions": {"Na": M["Na"], "HCO3": M_HCO3}},
}

def contribution_ppm_per_gL(salt_name):
    """Vector (len IONS): ppm-toename per 1 g/L toegevoegd."""
    s = salts[salt_name]
    mm = s["MM"]
    vec = np.zeros(len(IONS), dtype=float)
    for i,ion in enumerate(IONS):
        if ion in s["ions"]:
            mass_fraction = s["ions"][ion] / mm
            vec[i] = 1000.0 * mass_fraction
    return vec

salt_names = list(salts.keys())
A = np.vstack([contribution_ppm_per_gL(n) for n in salt_names]).T  # ions x zouten

for j,n in enumerate(salt_names):
    parts = [f"{IONS[i]} {A[i,j]:.1f}" for i in range(len(IONS)) if A[i,j]>0]
    print(f"{n:12s}: " + ", ".join(parts))

## 4. Optimalisatie: zoutdosissen vinden

We zoeken niet-negatieve dosissen \(a_j \ge 0\) (in g/L) zodat:

\[
c_{start} + A a \approx c^*
\]

We minimaliseren een gewogen kwadratische fout met **projected gradient descent** (projectie op a≥0).  
U kunt maxima opleggen om realistische grenzen te respecteren.


In [ ]:
def dict_to_vec(d, ions=IONS):
    return np.array([float(d.get(k,0.0)) for k in ions], dtype=float)

def projected_nonneg_opt(A, b, w, a0=None, lr=2e-4, steps=18000, a_max=None):
    """Minimize sum_i w_i (A a - b)_i^2  s.t. a>=0 (en optioneel a<=a_max)."""
    m,n = A.shape
    a = np.zeros(n, dtype=float) if a0 is None else np.maximum(np.array(a0, dtype=float), 0.0)

    if a_max is None:
        a_max = np.full(n, np.inf, dtype=float)
    else:
        a_max = np.array(a_max, dtype=float)

    hist = []
    checkpoints = max(1, steps//240)

    for k in range(steps):
        r = (A @ a - b)
        grad = 2.0 * (A.T @ (w * r))
        a = a - lr * grad
        a = np.clip(a, 0.0, a_max)

        if k % checkpoints == 0:
            hist.append(float(np.sum(w * (A @ a - b)**2)))

    return a, np.array(hist, dtype=float)

c0 = dict_to_vec(start_water)
ct = dict_to_vec(target)
w  = dict_to_vec(weights)
b  = ct - c0

a_max = {
    "CaCl2·2H2O": 1.5,
    "CaSO4·2H2O": 2.0,
    "MgSO4·7H2O": 1.0,
    "NaCl":       1.0,
    "NaHCO3":     1.0,
}
a_max_vec = np.array([a_max[n] for n in salt_names], dtype=float)

a_opt, hist = projected_nonneg_opt(A, b, w, lr=2e-4, steps=18000, a_max=a_max_vec)
c_new = c0 + A @ a_opt

print("=== Zoutdosissen (g/L) ===")
for n,aj in zip(salt_names, a_opt):
    print(f"{n:12s}: {aj:7.4f} g/L  |  {aj*20:7.3f} g per 20 L")

print("\n=== Start → nieuw (ppm) vs doel ===")
for ion,i in zip(IONS, range(len(IONS))):
    print(f"{ion:>5s}: {c0[i]:7.1f}  → {c_new[i]:7.1f}   (doel {ct[i]:.1f})")

print(f"\nGewogen fout J: {float(np.sum(w*(c_new-ct)**2)):.3f}")

## 5. Visualisatie

- Balkplot: start vs. na zouten vs. doel  
- Convergentieplot: kostfunctie doorheen iteraties  
- Cl/SO₄ ratio (indicatief)


In [ ]:
x = np.arange(len(IONS))

plt.figure(figsize=(10,4.8))
plt.bar(x-0.28, c0,    width=0.26, label="Start")
plt.bar(x,      c_new, width=0.26, label="Na zouten")
plt.bar(x+0.28, ct,    width=0.26, label="Doel")
plt.xticks(x, IONS)
plt.ylabel("mg/L (ppm)")
plt.title(f"Zouttoevoegingen – stijl: {chosen_style}")
plt.grid(True, axis="y", alpha=0.25)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7.5,4.2))
plt.plot(hist)
plt.xlabel("Checkpoints (subsamples van iteraties)")
plt.ylabel("Gewogen fout J")
plt.title("Convergentie (projected nonnegative optimizer)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

def safe_ratio(cl, so4):
    return np.nan if so4 <= 1e-9 else cl/so4

r0 = safe_ratio(c0[IONS.index("Cl")],  c0[IONS.index("SO4")])
r1 = safe_ratio(c_new[IONS.index("Cl")], c_new[IONS.index("SO4")])
rt = safe_ratio(ct[IONS.index("Cl")],  ct[IONS.index("SO4")])

print(f"Cl/SO4 ratio – start: {r0:.2f} | na zouten: {r1:.2f} | doel: {rt:.2f}")

## 6. Interpretatievragen

1. Kies **Pilsner** en daarna **Malt-forward**. Welke zouten veranderen vooral? Waarom?  
2. Zet `a_max["NaHCO3"]=0.0` (geen baksoda). Wat gebeurt er met HCO₃ en met de totale fout?  
3. Maak HCO₃ minder belangrijk (`weights["HCO3"]=0.2`). Wat is het effect op de zoutkeuze?  
4. Welk zout beïnvloedt zowel Ca als Cl? En zowel Ca als SO₄? Geef de chemische verklaring.  
5. Welke extra real-world effecten ontbreken (neerslag, pH/CO₂, oplosbaarheid)? Hoe zou u dit modelleren?


## 7. Uitbreidingen (optioneel)

- Startwater uit **Notebook 3** (blend-output) rechtstreeks inlezen.  
- Range-targets (min/max per ion) i.p.v. één puntdoel.  
- Koppeling met **mash pH** (Notebook 1) via alkaliniteit/zuurcorrectie.
